In [1]:
from chatzhipuai import *
ZHIPUAI_API_KEY = "xxx"
llm = ChatZhipuAI(
    temperature=0.1,
    api_key=ZHIPUAI_API_KEY,
    model_name="glm-4",
)

In [2]:
import sys
print(sys.path)

['/home/ecd_bladelee_1085274/proj/src/workbench/llm', '/home/ecd_bladelee_1085274/miniconda3/envs/langchain/lib/python39.zip', '/home/ecd_bladelee_1085274/miniconda3/envs/langchain/lib/python3.9', '/home/ecd_bladelee_1085274/miniconda3/envs/langchain/lib/python3.9/lib-dynload', '', '/home/ecd_bladelee_1085274/miniconda3/envs/langchain/lib/python3.9/site-packages']


In [4]:
llm.invoke("langsmith如何帮助测试?")

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是世界级的技术文档作者。"),
    ("user", "{input}")
])
chain = prompt | llm
chain.invoke({"input": "langsmith如何帮助测试?"})


from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是世界级的技术文档作者。"),
    ("user", "{input}")
])

output_parser = StrOutputParser()
chain = prompt | llm | output_parser
chain.invoke({"input": "langsmith如何帮助测试?"})


from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""仅根据所提供的上下文回答以下问题:

<context>
{context}
</context>

问题: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

from langchain.chains import create_retrieval_chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "langsmith如何帮助测试?"})
print(response["answer"])

LangSmith 提供了评估功能来帮助测试语言模型 (LLM) 应用程序。这包括内置的准确性评估器，您可以使用它来定义测试用例和评估标准。以下是使用 LangSmith 进行测试的一些步骤：

1. **创建数据集**：首先，您需要定义一个数据集，其中包含用于测试的输入和预期输出。在 LangSmith 中，您可以使用 `create_dataset` 方法来创建一个数据集，并使用 `create_examples` 方法来定义数据集中的具体测试用例。

2. **定义评估器**：接下来，您可以定义一个评估器来评估您的 LLM 应用程序的输出是否符合预期。例如，您可以创建一个评估器来检查输出的确切匹配。

3. **进行评估**：使用 `evaluate` 方法，您可以运行一个评估实验，该方法接受您的 LLM 应用程序作为输入，数据集作为测试用例，以及评估器来评分结果。

4. **查看评估结果**：评估完成后，您可以查看实验结果，以了解您的 LLM 应用程序在测试用例上的表现。

通过这些步骤，LangSmith 允许您在实际部署前对 LLM 应用程序进行全面的测试，确保其达到预期的性能标准。


In [6]:

## simple  agent example

from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "搜索有关LangSmith的信息。关于LangSmith的任何问题，您都可以使用这个工具",
)

from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()
tools = [retriever_tool, search]
from langchain import hub
from langchain.agents import create_openai_tools_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "langsmith如何帮助测试?"})

ValidationError: 1 validation error for TavilySearchAPIWrapper
__root__
  Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. (type=value_error)